In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [4]:
%%nuclio cmd
pip install kfserving
pip install numpy
pip install xgboost

    100% |████████████████████████████████| 61kB 2.7MB/s ta 0:00:011
    100% |████████████████████████████████| 92kB 7.9MB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 30.8MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 19.1MB/s ta 0:00:01


In [2]:
import kfserving
import os
import numpy as np
from typing import List, Any

import xgboost as xgb

In [3]:
BOOSTER_FILE = "model.bst"

class XGBoostModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: xgb.XGBModel = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not booster is None:
            self._booster = booster
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), BOOSTER_FILE)
        self._booster = xgb.Booster(model_file=model_file)
        self.ready = True

    def predict(self, body: List) -> List:
        try:
            # Use of list as input is deprecated see https://github.com/dmlc/xgboost/pull/3970
            dmatrix = xgb.DMatrix(body)
            result: xgb.DMatrix = self._booster.predict(dmatrix)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)


In [2]:
# nuclio: end-code
from mlrun import new_model_server
import requests

In [3]:
fn = new_model_server('mysrv2', models={'netops_v1': 'file:///User/mlrun/models'}, model_class='XGBoostModel')
fn.with_v3io('User','~/') 

In [4]:
addr = fn.deploy(project='xgb')

[nuclio.deploy] 2019-09-21 18:57:43,104 (info) Building processor image
[nuclio.deploy] 2019-09-21 18:57:48,145 (info) Build complete
[nuclio.deploy] 2019-09-21 18:57:54,198 (info) Function deploy complete
[nuclio.deploy] 2019-09-21 18:57:54,204 done updating mysrv2, function address: 18.221.60.238:30585


In [5]:
# Seldon protocol event
event_seldon = {"data": {"ndarray": [[5], [10]]}}
csel = str(event_seldon).replace("\'", "\"")

In [6]:
resp = requests.put(addr + '/predict/netops_v1', data=csel)
print(resp.content)

b'{"data": {"ndarray": [[0.9823893308639526, 0.003680035937577486, 0.0034835119731724262, 0.0014924356946721673, 0.0014924356946721673, 0.0014924356946721673, 0.0014924356946721673, 0.0014924356946721673, 0.0014924356946721673, 0.0014924356946721673], [0.976058304309845, 0.009153352119028568, 0.004408582113683224, 0.0014828175771981478, 0.0014828175771981478, 0.0014828175771981478, 0.0014828175771981478, 0.0014828175771981478, 0.0014828175771981478, 0.0014828175771981478]]}}'


In [ ]:
# convert to KubeFlow Pipelines step (which deploy the function)
# should be inside a pipeline DSL 
fn.deploy_step()

In [ ]:
fn.to_dict()